In [84]:
import numpy as np
from scipy.stats import norm

## Problem 1

## (a)

I choose to manually fill in the covariance matrix. Since $X^{[j]} = log S^{[j]}$, then, $dX_t^{[j]} = rdt + \sigma_{[j]dW_t^{[j]}}$. 

\begin{align*}
 cov(X_T^1,X_T^2) &= cov(log S_0^1 + \int^T_0 r dt + \int^T_0 \sigma_1 dW_t^1,log S_0^2 + \int^T_0 r dt + \int^T_0 \sigma_2 dW_t^2 ) \\
& = cov ( \int^T_0 \sigma_1 dW_t^1,  \int^T_0 \sigma_2 dW_t^2) = cov(\sigma_1W_T^1, \sigma_2W_T^2)\\
& = \sigma_1\sigma_2 cov(W_T^1,W_T^2) = 0.3 \times 0.2 \times 0.8 T  = 0.0048T \\
& cov(X_T^1,X_T^1) = 0.09T \\
& cov(X_T^2,X_T^1) = 0.0048T\\
& cov(X_T^2,X_T^2) = 0.04T\\
\end{align*}

## (b)

We generate the correlated Brownian motion using the formula $LL^T = H$. And $X_T^{m} = X_0 = \mu T + L \sqrt{T}Z^{m}$

In [13]:
class MultiGBM:

    def __init__(self,S0,r,correlations,sigma):
        self.S0 = S0
        self.r = r
        self.correlations = correlations
        self.sigma = sigma

In [14]:
hw6p1dynamics = MultiGBM(S0=np.array([100,110]),r=0.05,
                         correlations = np.array([[1,0.8],[0.8,1]]),
                         sigma = np.diag([0.3, 0.2]))

In [16]:
class CallOnBasket:

    def __init__(self,K,T,weights):
        self.K = K
        self.T = T
        self.weights = weights

In [17]:
hw6p1contract=CallOnBasket(K=110,T=1.0,weights = np.array([1/2, 1/2]))

In [89]:
class MCengine:

    def __init__(self, M, antithetic, control, seed):
        self.M = M                                   
        self.antithetic = antithetic
        self.control = control
        self.rng = np.random.default_rng(seed=seed)  

    def price_callonbasket_multiGBM(self,contract,dynamics):

        L =  np.linalg.cholesky(dynamics.correlations)
        sigma = dynamics.sigma
        S0 = dynamics.S0
        r = dynamics.r

        T = contract.T
        K = contract.K
        weights = contract.weights

        X_0 = np.ones((self.M, len(S0)))
        X_0 = X_0 * np.log(S0)
        X_0 = X_0.T
        sigma_mat = (np.ones((self.M, len(S0))) * np.diag(sigma)).T
        
        dW_T = self.rng.multivariate_normal([0,0],dynamics.correlations,size = self.M)  * np.sqrt(T) 
        X_T = X_0 + (r-sigma_mat**2/2)*T + sigma.dot(L.dot(dW_T.T))

        S_T = np.exp(X_T)
        H_T = weights.dot(S_T)
        C_T = np.exp(-r*T) * np.maximum(H_T-K, 0)

        if self.antithetic: 
            X_T_a = X_0 + (r-sigma_mat**2/2)*T - sigma.dot(L.dot(dW_T.T))
            S_T_a = np.exp(X_T_a)
            H_T_a = weights.dot(S_T_a)
            C_T_a = np.exp(-r*T) * np.maximum(H_T_a-K, 0)
            C_T = (C_T + C_T_a)/2

        def call_price_BS_formula(sigma, S, rGrow, r, T, K):
            F= S*np.exp(rGrow*T)
            sd = sigma*np.sqrt(T)
            d1 = np.log(F/K)/sd+sd/2
            d2 = d1-sd
            return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))
        call_price = np.mean(C_T)
        standard_error = np.std(C_T, ddof=1)/np.sqrt(self.M)
    

        if self.control: 
            # \frac{\sqrt{(\sigma_1^2 + \sigma^2_2 + 2\rho\sigma_1\sigma_2)}}{2} 
            sigma_c = np.sqrt(sigma[0,0]**2 + sigma[1,1]**2 + 2*sigma[0,0]*sigma[1,1]*dynamics.correlations[0,1])/2
            S_c = (S0[0]*S0[1])**0.5
            #r + \frac{-\sigma^2_1-\sigma^2_2  + 2\rho\sigma_1\sigma_2}{8}
            rGrow_c = r + (2*sigma[0,0]*sigma[1,1]*dynamics.correlations[0,1] - sigma[0,0]**2 - sigma[1,1]**2)/8
            
            C_BS = call_price_BS_formula(sigma_c, S_c, rGrow_c, r, T, K)

            EY = np.exp(np.mean(X_T, axis = 0))
            C_c = np.exp(-r*T) * np.maximum(EY-K, 0)
            cov = np.cov(C_T, C_c)
            beta = cov[0,1] / cov[1,1]

            Y_c = C_T + beta*(C_BS - C_c)
            call_price = np.mean(Y_c)
            standard_error = np.std(Y_c, ddof=1) / np.sqrt(self.M)

        return(call_price, standard_error)

In [79]:
hw6p1bMC=MCengine(M=10000,antithetic=False,control=False,seed=0)
(call_price_ordinary, std_err_ordinary) = hw6p1bMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_ordinary, std_err_ordinary)

12.12467498776292 0.20758555753501975


The call price is approximately 12.12, and the standard error is approximately 0.2076. 

## (c)

In [83]:
hw6p1cMC=MCengine(M=10000,antithetic=True,control=False,seed=0)
(call_price_AV, std_err_AV) = hw6p1cMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_AV, std_err_AV)

12.214537629057864 0.11712125008722657


The call price under atithetic variance reduction is approximately 12.21, and the standard error is approximately 0.1171. 

## (d)

\begin{align*}
& E[log G_T] = E[log (\frac{1}{2}S_1 S_2)] = 1/2E[log S_1] + 1/2 E[log S_2] \\
& = 1/2[log(S_0^{[1]}) + (r- 1/2 \sigma_1^2)T + log(S_0^{[2]}) + (r - 1/2 \sigma_2^2)T] \\ 
& = \frac{1}{2} log(S_0^{[1]}S_0^{[2]}) (r- \frac{\sigma^2_1+\sigma^2_2}{4})T \\

& Var[log G_T] = 1/4Var[log S_1 + log S_2] = 1/4[(var(log S_1) + var(log S_2) + 2 cov(log S_1, log S_2))T] \\
& = \frac{(\sigma_1^2 + \sigma^2_2 + 2\rho\sigma_1\sigma_2)T}{4}
\end{align*}

## (e)

According to lec 6 Finm330: $C^G = C(G_0,0,K,T,R_{grow},r,\sigma)$.

$r = R_{grow} - 1/2 \sigma^2$

$R_{grow} = r + 1/2 \sigma^2$

$r + \sigma^2 /2 = r- \frac{\sigma^2_1+\sigma^2_2}{4} + \frac{(\sigma_1^2 + \sigma^2_2 + 2\rho\sigma_1\sigma_2)}{8}  = r + \frac{-\sigma^2_1-\sigma^2_2  + 2\rho\sigma_1\sigma_2}{8}$

$C^G = C^{BS} ((S_0^{[1]}S_0^{[2]})^{1/2},0,K,T,r + \frac{-\sigma^2_1-\sigma^2_2  + 2\rho\sigma_1\sigma_2}{8},r,\frac{\sqrt{(\sigma_1^2 + \sigma^2_2 + 2\rho\sigma_1\sigma_2)}}{2} )$

## (f)

In [90]:
hw6p1fMC=MCengine(M=10000,antithetic=False,control=True,seed=0)
(call_price_CV, std_err_CV) = hw6p1fMC.price_callonbasket_multiGBM(hw6p1contract,hw6p1dynamics)
print(call_price_CV, std_err_CV)

9.889660529217075 0.00203055502895262


The call price is roughly 9.89, and the standard error is approximately 0.0020

## Problem 2

In [91]:
class GBM:

    def __init__(self,sigma,r,S0):
        self.sigma = sigma
        self.r = r
        self.S0 = S0

In [92]:
hw6p2dynamics=GBM(sigma=0.2,r=0.02,S0=100)

In [93]:
class CallOption:

    def __init__(self,K,T):
        self.K=K
        self.T=T

In [94]:
hw6p2contract=CallOption(K=150,T=1)

In [105]:
class MCimportanceEngine:

    def __init__(self, M, lamb, seed):
        self.M = M                                  
        self.lamb = lamb                            
        self.rng = np.random.default_rng(seed=seed) 

    def price_call_GBM(self, contract,dynamics):

        S0 = dynamics.S0
        r = dynamics.r
        sigma = dynamics.sigma
        T = contract.T
        K = contract.K
        Z = self.rng.normal(size = self.M)

        X_T = np.log(S0) + (r - sigma**2/2)*T + sigma*np.sqrt(T)*(Z+self.lamb*T)
        S_T = np.exp(X_T)
        C_T = np.exp(-r*T-self.lamb*Z -0.5*(self.lamb**2*T)) * np.maximum(S_T-K, 0)

        call_price = np.mean(C_T)
        standard_error = np.std(C_T, ddof=1)/np.sqrt(self.M)

        return(call_price, standard_error)


## (a)

In [106]:
hw6p2aMC=MCimportanceEngine(M=100000,lamb=0,seed=0) 
(call_price_ordinary, std_err_ordinary) =  hw6p2aMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print(call_price_ordinary, std_err_ordinary)

0.25270332833609405 0.007609293292996182


The price of vanilla call option is 0.2527, and the standard error is approximatly 0.0076.

## (b)

According to L6.21, $E[S_T]= S_0 e^{(r+\sigma \lambda)T}$.

$165 = 100 exp(0.02 + 0.2\lambda)$

$\lambda = 2.40387$


## (c)

In [107]:
hw6p2cMC=MCimportanceEngine(M=100000,lamb= 2.40387,seed=0) # Fill in the lamb parameter with the lambda that you compute in (b)
(call_price_importsamp, std_err_importsamp) =  hw6p2cMC.price_call_GBM(hw6p2contract,hw6p2dynamics)
print(call_price_importsamp, std_err_importsamp)

0.2484366181965453 0.000773428279853675


The price of vanilla call option is 0.2484, and the standard error is approximatly 0.00077.